<a href="https://colab.research.google.com/github/bforoura/Transformers/blob/main/nlp_transformers_ch2_ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Activating the GPU
# Main menu->Runtime->Change Runtime Type

import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')

print('Found GPU at: {}'.format(device_name))


Found GPU at: /device:GPU:0


In [4]:
#@title Installing the Hugging Face PyTorch Interface for Bert

!pip install -q transformers



In [5]:
#@title Importing the modules

import torch

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from transformers import BertTokenizer, BertConfig
from transformers import AdamW, BertForSequenceClassification, get_linear_schedule_with_warmup


In [6]:
from tqdm import tqdm, trange

import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt



In [7]:
#@title Specify CUDA as device for Torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

torch.cuda.get_device_name(0)


'Tesla T4'

In [14]:
#@title Loading the Dataset


df = pd.read_csv("https://raw.githubusercontent.com/PacktPublishing/Transformers-for-Natural-Language-Processing/main/Chapter02/in_domain_train.tsv", 
                 delimiter='\t', header=None, 
                 names=['sentence_source', 'label', 'label_notes', 'sentence'])

df.shape


(8551, 4)

In [15]:
df.sample(10)


,sentence_source,label,label_notes,sentence
2874,l-93,1,NaN,herman mixed the eggs and the cream together .
2223,l-93,1,NaN,that dog bites people .
4596,ks08,1,NaN,i did n't find any bugs in my bed .
5337,b_73,1,NaN,jack eats more caviar than he eats mush .
3895,ks08,0,*,john placed behind the counter .
1997,rhl07,1,NaN,martha gave myrna an apple .
6081,c_13,1,NaN,calvin was eating .
3625,ks08,1,NaN,john looked up the meaning of ` chanson ' .
346,bc01,0,*,john write books .
1578,r-67,0,*,"after ten soldiers had left , seven more ones ..."
